In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
os.chdir("..")
print(os.getcwd())
sys.path.insert(0, os.getcwd())

/home/gy46/NFGP


In [2]:
import trimesh
import matplotlib
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt

# Visualize the Implicit Field

In [3]:
!ls -d logs/*/

logs/bar_twist_s1e-1_b1e-3_2021-Nov-26-17-52-13/
logs/bar_twist_s1e-1_b1e-5_2021-Nov-28-12-13-04/
logs/cactus_translate_s1e-1_b1e-3_2021-Nov-28-12-13-06/
logs/cactus_xrotate_s1e-1_b1e-3_2021-Nov-28-12-13-11/
logs/cactus_yrotate_s1e-1_b1e-3_2021-Nov-28-12-13-12/
logs/checkpoints/
logs/cylinder_rotate_s1e-1_b1e-3_2021-Nov-23-09-45-21/
logs/cylinder_translate_s1e-1_b1e-3_2021-Nov-23-09-45-39/
logs/dino_bend_s0_b1e-3_2021-Nov-21-16-47-21/
logs/dino_bend_s1e-1_b0_2021-Nov-21-16-47-17/
logs/dino_bend_s1e-1_b1e-3_2021-Nov-21-16-43-33/
logs/dino_bend_s1e-1_b1e-3_2021-Nov-23-09-45-52/
logs/dino_bend_s1e-1_b1e-3_hthr1e-4_2021-Nov-21-16-47-25/
logs/images/
logs/val/
logs/vis/


In [9]:
# log_path = "logs/cylinder_rotate_s1e-1_b1e-3_2021-Nov-23-09-45-21/"
log_path = "logs/bar_twist_s1e-1_b1e-5_2021-Nov-28-12-13-04/"

In [10]:
import os.path as osp
from utils import load_imf

net, cfg = load_imf(
    log_path, 
    return_cfg=True)
net, cfg

/home/gy46/NFGP/utils.py:51: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = dict2namespace(yaml.load(f))


logs/bar_twist_s1e-1_b1e-5_2021-Nov-28-12-13-04/checkpoints/epoch_999_iters_100000.pt
Original Decoder:
Net(
  (blocks): ModuleList(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=1, bias=True)
  )
  (act): Sine()
)
Net(
  (blocks): ModuleList(
    (0): InvertibleResBlockLinear(
      (pos_enc): LipBoundedPosEnc()
      (blocks): ModuleList(
        (0): Linear(in_features=33, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=256, bias=True)
        (2): Linear(in_features=256, out_features=3, bias=True)
      )
      (act): ELU(alpha=1.0)
    )
    (1): InvertibleResBlockLinear(
      (pos_enc): LipBoundedPosEnc()
      (blocks): ModuleList(
        (0): Linear(in_features=33, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=256, bias=True)
      

100%|██████████| 1678/1678 [00:04<00:00, 405.28it/s]


(DeformationWrapper(
   (orig): Net(
     (blocks): ModuleList(
       (0): Linear(in_features=3, out_features=512, bias=True)
       (1): Linear(in_features=512, out_features=512, bias=True)
       (2): Linear(in_features=512, out_features=512, bias=True)
       (3): Linear(in_features=512, out_features=1, bias=True)
     )
     (act): Sine()
   )
   (deform): Net(
     (blocks): ModuleList(
       (0): InvertibleResBlockLinear(
         (pos_enc): LipBoundedPosEnc()
         (blocks): ModuleList(
           (0): Linear(in_features=33, out_features=256, bias=True)
           (1): Linear(in_features=256, out_features=256, bias=True)
           (2): Linear(in_features=256, out_features=3, bias=True)
         )
         (act): ELU(alpha=1.0)
       )
       (1): InvertibleResBlockLinear(
         (pos_enc): LipBoundedPosEnc()
         (blocks): ModuleList(
           (0): Linear(in_features=33, out_features=256, bias=True)
           (1): Linear(in_features=256, out_features=256, bias=Tr

# Load dataset

In [11]:
import trimesh
import importlib
data_lib = importlib.import_module(cfg.data.type)
loader = data_lib.get_data_loaders(cfg.data, None)['train_loader']
for data in loader:
    break
    
print(data.keys())
handles = data['handles'].view(-1, 3).detach().cpu().numpy()
targets = data['targets'].view(-1, 3).detach().cpu().numpy()
print(handles.shape, targets.shape)


p = mp.plot(data['gtr_verts'].cpu().view(-1, 3).numpy(), 
        data['gtr_faces'].cpu().view(-1, 3).numpy())
p.add_points(handles, shading={'point_color': 'red', 'point_size': 0.05})
p.add_points(targets, shading={'point_color': 'blue', 'point_size': 0.05})

dict_keys(['idx', 'handles', 'targets', 'gtr_verts', 'gtr_faces'])
(996, 3) (996, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000789…

2

# Get the output Mesh

In [12]:
from trainers.utils.vis_utils import imf2mesh
res = 256
out_mesh = imf2mesh(net, res=res, normalize=True, bound=1.2)

100%|██████████| 1678/1678 [00:18<00:00, 88.67it/s]


In [13]:
p = mp.plot(out_mesh.vertices.astype(np.float32), out_mesh.faces.astype(np.float32)) 
p.add_points(handles, shading={'point_color': 'red', 'point_size': 0.05})
p.add_points(targets, shading={'point_color': 'blue', 'point_size': 0.05})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.070613…

2

In [ ]:
# Visualize the original neural fields